#Analisador de Processo de Compra

Objetivo 1: Construir as mensagens dos emails para envio das notas de empenho

Objetivo 2: Listar os produtos fracassados e checar se todos os produtos homologados pela DG constam nas notas de empenho anexadas ao processo

Objetivo 3: Criar uma Planilha para acompanhamento das entregas dos produtos pelos fornecedores

Objetivo 4: Checar se todos os pagamentos aos fornecedores já foram feitos para poder arquivar o processo

In [567]:
!pip install nltk
!pip install PyPDF2



In [568]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [569]:
import PyPDF2

medicamentos = 'SEI_03789_2024.pdf'
mat_hosp_odon = 'SEI_03699_2024.pdf'

# Abrir o arquivo PDF
with open(f'/content/drive/MyDrive/Colab Notebooks/Processo_compras/{mat_hosp_odon}', 'rb') as file:
    reader = PyPDF2.PdfReader(file)

    # Extrair texto de todas as páginas
    textPDF = ''
    for page in reader.pages:
        textPDF += page.extract_text()

    # Obter o número de páginas
    num_pages = len(reader.pages)

# Exibir o número de páginas
print(f"O documento contém {num_pages} páginas.")

O documento contém 1467 páginas.


O processo é carregado para textPDF

In [570]:
#capturar os dados principais das notas de empenho

import nltk
nltk.download('punkt')
# Dividir o texto em linhas
lines = nltk.tokenize.sent_tokenize(textPDF)

notas_de_empenho = [line for line in lines if "Ano Tipo Número70008 TRIBUNAL REGIONAL ELEITORAL DO RIO G.DO NORTE REAL" in line]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [571]:
import pprint

# Criar o dicionário
lista_dict = []
data_dict = {}

for nota_emp in notas_de_empenho:
  # Extrair as informações
  lines = nota_emp.splitlines()

  # Criar o dicionário
  data_dict = {}

  # Extrair as informações
  lines = nota_emp.splitlines()

  # Encontrar e adicionar 'NE'
  for line in lines:
      if "NE" in line:
          parts = line.split()
          data_dict['NE'] = parts[parts.index('NE') + 1]

  # Encontrar e adicionar 'Empresa' e 'CNPJ'
  for line in lines:
      if "/0001-" in line:
          parts = line.split()
          data_dict['CNPJ'] = parts[0]
          data_dict['Empresa'] = ' '.join(parts[1:])
          break  # Para evitar múltiplas capturas

  # Encontrar e adicionar 'Valor' usando referências
  for line in lines:
      if "0,0000" in line:
          # Capturar o valor que vem depois de "0,0000 "
          valor = line.split("0,0000 ")[1].split("\n")[0].strip()
          data_dict['Valor'] = valor
          break  # Para evitar múltiplas capturas

  # Adicionar o dicionário à lista
  lista_dict.append(data_dict)
# Exibir o dicionário
pprint.pprint(lista_dict)

[{'CNPJ': '54.416.857/0001-16',
  'Empresa': 'L & C PARTICIPACOES E COMERCIO LTDA',
  'NE': '332',
  'Valor': '9.862,81'},
 {'CNPJ': '43.492.384/0001-10',
  'Empresa': 'ANA C REGIS',
  'NE': '333',
  'Valor': '311,14'},
 {'CNPJ': '22.077.847/0001-07',
  'Empresa': 'JOSE DANTAS DINIZ FILHO',
  'NE': '334',
  'Valor': '381,75'},
 {'CNPJ': '19.367.167/0001-13',
  'Empresa': 'COMERCIAL TARGET DE EPI E EPC LTDA',
  'NE': '335',
  'Valor': '1.592,50'},
 {'CNPJ': '30.843.402/0001-19',
  'Empresa': 'DOIS K COMERCIO E SERVICOS LTDA',
  'NE': '336',
  'Valor': '6.491,60'},
 {'CNPJ': '43.492.384/0001-10',
  'Empresa': 'ANA C REGIS',
  'NE': '326',
  'Valor': '126,00'},
 {'CNPJ': '30.843.402/0001-19',
  'Empresa': 'DOIS K COMERCIO E SERVICOS LTDA',
  'NE': '348',
  'Valor': '380,00'},
 {'CNPJ': '30.843.402/0001-19',
  'Empresa': 'DOIS K COMERCIO E SERVICOS LTDA',
  'NE': '349',
  'Valor': '145,50'},
 {'CNPJ': '43.492.384/0001-10',
  'Empresa': 'ANA C REGIS',
  'NE': '350',
  'Valor': '515,80'},
 {

In [572]:
def remover_dicionarios_por_chave(lista_dicionarios, chave_desejada):
  """Remove da lista de dicionários todos os dicionários que não começam com NE (não são Notas de Empenho)
  """
  lista_filtrada = []
  for dicionario in lista_dicionarios:
    if chave_desejada in dicionario:
      lista_filtrada.append(dicionario)
  return lista_filtrada

lista_de_dicionarios = remover_dicionarios_por_chave(lista_dict, "NE")

In [573]:
#criação do DataFrame df_Notas_de_empenho com os dados do dicionario lista_dict
import pandas as pd
df_Notas_de_empenho = pd.DataFrame(lista_de_dicionarios)
df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81"
1,333,43.492.384/0001-10,ANA C REGIS,"311,14"
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75"
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50"
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60"
5,326,43.492.384/0001-10,ANA C REGIS,"126,00"
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00"
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50"
8,350,43.492.384/0001-10,ANA C REGIS,"515,80"
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95"


In [574]:
# capturar os emails que existirem no processo
import re

# Expressão regular para encontrar emails
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Encontrar todos os emails no texto
emails = re.findall(email_pattern, textPDF)
emails = list(set(emails))

# Remover os emails que forem do 'tre-rn'
filtered_emails = [item for item in emails if 'tre-rn' not in item]

email_minusculas = []
for string in filtered_emails:
    email_minusculas.append(string.lower())

email_minusculas = list(set(email_minusculas))
# Exibir os emails encontrados
print(email_minusculas)


['licitacoes@targethospitalar.com.br', 'lcparticipacoescomercio@gmail.com', 'acoescomercio@gmail.com', 'doiskcomercio@gmail.com', 'anacregis.parnamirim@gmail.com', 'nacregis.parnamirim@gmail.com', 'dantasdiniz@gmail.com']


In [575]:
#pegar os emails

import re

emails = []

for i in range (df_Notas_de_empenho.shape[0]):
  emails.append("nao_encontrado")

for i in range (df_Notas_de_empenho.shape[0]):
  # String a ser encontrada
  company_name = df_Notas_de_empenho['Empresa'][i]
  stop_phrase = 'Solicito análise das propostas'
 # stop_phrase = 'Validade da proposta'

  # Regex para encontrar um email
  email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
  # Encontrar a posição da frase que limita a busca
  stop_index = textPDF.find(stop_phrase)

  if stop_index != -1:
      # Obter o texto antes da frase de parada
      text_before_stop = textPDF[:stop_index]
      # Encontrar o nome da empresa no texto
      company_match_name = re.search(re.escape(company_name), text_before_stop)

      if company_match_name:
          # Obter o texto após a empresa
          text_after_company_name = text_before_stop[company_match_name.end():]
          # Procurar o primeiro email no texto subsequente
          email_match = re.search(email_pattern, text_after_company_name)
          if email_match:
              emails[i] = email_match.group()

  else:
      print("Frase de parada não encontrada no texto.")

for i in range (df_Notas_de_empenho.shape[0]):
  # String a ser encontrada
  company_cnpj = df_Notas_de_empenho['CNPJ'][i]
  stop_phrase = 'Solicito análise das propostas'

  # Regex para encontrar um email
  email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
  # Encontrar a posição da frase que limita a busca
  stop_index = textPDF.find(stop_phrase)

  if stop_index != -1:
      # Obter o texto antes da frase de parada
      text_before_stop = textPDF[:stop_index]

      # Encontrar o cnpj da empresa no texto
      company_match_cnpj = re.search(re.escape(company_cnpj), text_before_stop)

      if company_match_cnpj:
          # Obter o texto após a empresa
          text_after_company_cnpj = text_before_stop[company_match_cnpj.end():]
          # Procurar o primeiro email no texto subsequente
          email_match = re.search(email_pattern, text_after_company_cnpj)

          if email_match:
            if emails[i] == "nao_encontrado":
              emails[i] = email_match.group()


  else:
      print("Frase de parada não encontrada no texto.")

df_Notas_de_empenho['Email'] = emails



In [576]:
df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor,Email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",nacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",nao_encontrado
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",nacregis.parnamirim@gmail.com
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",nacregis.parnamirim@gmail.com
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM


In [577]:

# # Função para encontrar a linha do email no texto extraído
# def find_email_line(email):
#     # Regex para encontrar o email no texto extraído
#     match = re.search(re.escape(email), extracted_text)
#     if match:
#         # Obter a linha onde o email foi encontrado
#         lines = extracted_text.splitlines()
#         for line in lines:
#             if email in line:
#                 return line  # Retorna a linha correspondente
#     return None  # Retorna None se o email não for encontrado

# # Criar nova coluna 'Verif_email' com as linhas correspondentes
# df_Notas_de_empenho['Verif_email'] = df_Notas_de_empenho['Email'].apply(find_email_line)

# # Exibir o DataFrame atualizado

# df_Notas_de_empenho

In [578]:
# lista_empresas_sem_email = []
# cnjp_empresas_sem_email = []
# for i in range (df_Notas_de_empenho.shape[0]):
#   if df_Notas_de_empenho['Email'][i] == "nao_encontrado":
#     lista_empresas_sem_email.append(df_Notas_de_empenho['Empresa'][i])
#     cnpj_limpo = df_Notas_de_empenho['CNPJ'][i].replace('.', '').replace('/', '').replace('-', '')
#     cnjp_empresas_sem_email.append(cnpj_limpo)
#     df_Notas_de_empenho['Verif_email'][i] = f'https://cnpj.biz/{cnpj_limpo}'

# print("Os emails das seguintes empresas não foram encontrados:")
# print(lista_empresas_sem_email)
# print()
# print("Na coluna 'Verif_email' do Dataframe foi gerado um link onde é possível encontrar o email")
# df_Notas_de_empenho

In [579]:
from IPython.display import display, HTML

# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Antes de continuar, cheque os emails na coluna do Email do DataFrame com os da lista abaixo.</h1>'
text2 = '<h1 style="color: red;">Caso seja necessário faça a correção dos emails.</h1>'
text3 = '<h1 style="color: red;">Foi colocado na célula abaixo um print errado para o código parar de executar.</h1>'
# Exibir o texto
# display(HTML(text3))
display(HTML(text1))
display(HTML(text2))

In [580]:
# print("Os e-mails encontrados np texto foram:")
# print(email_minusculas)
# print()
# pergunta = input("Deseja corrigir(inserir) manualmente algum email? (s/n): ")
# if pergunta.lower() == 's':
#   print(parar)

In [581]:
for i in range (len(email_minusculas)):
  print(f"{i+1}: {email_minusculas[i]}")

1: licitacoes@targethospitalar.com.br
2: lcparticipacoescomercio@gmail.com
3: acoescomercio@gmail.com
4: doiskcomercio@gmail.com
5: anacregis.parnamirim@gmail.com
6: nacregis.parnamirim@gmail.com
7: dantasdiniz@gmail.com


In [582]:
emails_verificados = []
for i in range (df_Notas_de_empenho.shape[0]):
  if df_Notas_de_empenho['Empresa'][i] not in emails_verificados:
    print(f"O email encontrado para a empresa {df_Notas_de_empenho['Empresa'][i]} é o email {df_Notas_de_empenho['Email'][i]}")
    # resposta = input("Deseja alterar o email? (s/n)")
    # if resposta.lower() == 's':
    print(f"Selecione no print acima, pelo número, o email para setar na Empresa: {df_Notas_de_empenho['Empresa'][i]}")
    n = int(input("Digite o número (0 se não deseja corrigir): "))
    emails_verificados.append(df_Notas_de_empenho['Empresa'][i])
    if n > 0:
        df_Notas_de_empenho.loc[df_Notas_de_empenho['Empresa'] == df_Notas_de_empenho['Empresa'][i], 'Email'] = email_minusculas[n-1]
    # else:
    #     df_Notas_de_empenho.loc[df_Notas_de_empenho['Empresa'] == df_Notas_de_empenho['Empresa'][i], 'Email'] = 'nao_encontrado'



df_Notas_de_empenho

O email encontrado para a empresa L & C PARTICIPACOES E COMERCIO LTDA é o email LCPARTICIPACOESCOMERCIO@GMAIL.COM
Selecione no print acima, pelo número, o email para setar na Empresa: L & C PARTICIPACOES E COMERCIO LTDA
Digite o número (0 se não deseja corrigir): 0
O email encontrado para a empresa ANA C REGIS é o email nacregis.parnamirim@gmail.com
Selecione no print acima, pelo número, o email para setar na Empresa: ANA C REGIS
Digite o número (0 se não deseja corrigir): 5
O email encontrado para a empresa JOSE DANTAS DINIZ FILHO é o email nao_encontrado
Selecione no print acima, pelo número, o email para setar na Empresa: JOSE DANTAS DINIZ FILHO
Digite o número (0 se não deseja corrigir): 7
O email encontrado para a empresa COMERCIAL TARGET DE EPI E EPC LTDA é o email licitacoes@targethospitalar.com.br
Selecione no print acima, pelo número, o email para setar na Empresa: COMERCIAL TARGET DE EPI E EPC LTDA
Digite o número (0 se não deseja corrigir): 0
O email encontrado para a empres

,NE,CNPJ,Empresa,Valor,Email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",anacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",anacregis.parnamirim@gmail.com
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",anacregis.parnamirim@gmail.com
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM


In [583]:
#para inserir manualmente digitando algum email que não tenha sido capturado

empresas_nao_encontrado = df_Notas_de_empenho.loc[df_Notas_de_empenho['Email'] == 'nao_encontrado', 'Empresa']
if len(empresas_nao_encontrado) > 0:
  empresas_nao_encontrado.reset_index(drop=True, inplace=True)
  print("Os email das seguinte empresas não foram inseridos:")
  print(empresas_nao_encontrado)
  for i in range (len(empresas_nao_encontrado)):
    email_para_inserir = input(f"Digite o email da empresa {empresas_nao_encontrado[i]} para inserção manual: ")
    df_Notas_de_empenho.loc[df_Notas_de_empenho['Empresa'] == empresas_nao_encontrado[i], 'Email'] = email_para_inserir

dropar = ['Verif_email']

df_Notas_de_empenho

Os email das seguinte empresas não foram inseridos:
0    INFINITI EMPREENDIMENTOS LTDA
Name: Empresa, dtype: object
Digite o email da empresa INFINITI EMPREENDIMENTOS LTDA para inserção manual: infiniti.licitacoes@gmail.com


,NE,CNPJ,Empresa,Valor,Email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",anacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",anacregis.parnamirim@gmail.com
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",anacregis.parnamirim@gmail.com
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM


Início de criação das mensagens para envio das Notas de Empenho

In [545]:

empresas = df_Notas_de_empenho['Empresa'].unique()  # Obter valores únicos da coluna "Empresa"

# Criar um dicionário com listas vazias, onde as chaves são os nomes das empresas
listas_empresas = {empresa: [] for empresa in empresas}

# Iterar sobre o DataFrame e preencher as listas com os números da coluna "NE"
for _, row in df_Notas_de_empenho.iterrows():
    empresa = row['Empresa']
    ne = row['NE']
    listas_empresas[empresa].append(ne)

# Exibir o dicionário preenchido
print(listas_empresas)



{'L & C PARTICIPACOES E COMERCIO LTDA': ['332', '351'], 'ANA C REGIS': ['333', '326', '350'], 'JOSE DANTAS DINIZ FILHO': ['334'], 'COMERCIAL TARGET DE EPI E EPC LTDA': ['335'], 'DOIS K COMERCIO E SERVICOS LTDA': ['336', '348', '349'], 'INFINITI EMPREENDIMENTOS LTDA': ['352']}


In [547]:
from datetime import datetime

# Obter a data atual
current_date = datetime.now()

# Formatar o dia, mês e ano como strings
formatted_date = f"SAMS - TRE-RN, {current_date.day}/{current_date.month}/{current_date.year}."

# Função para criar e-mails
def criar_emails(empresas):
    emails = []
    for empresa, numeros in empresas.items():
        for numero in numeros:

            num_NE = f'{current_date.year}NE000{numero}'
            email = f"""Prezados(as) Senhores(as),

Encaminhamos a Nota de Empenho número {num_NE} (em anexo), à empresa {empresa}.
Informamos que o prazo para entrega do material é de até 30 (trinta) dias corridos, contados a partir da data de envio
desta Nota de Empenho.
Solicitamos especial atenção quanto ao PRAZO, LOCAL e HORÁRIO para entrega do material.
A entrega deverá ser realizada na Seção de Assistência Médica e Saúde Ocupacional (SAMS), localizada na Sede
Administrativa do TRE/RN. Endereço: Av. Rui Barbosa, 215. Tirol, Natal-RN. CEP: 59.015-290. Telefone: (84) 3654-
5399, nos horários de segunda a quinta-feira, das 13 às 18 horas ou na sexta-feira, das 8 às 13 horas.
FAVOR CONFIRMAR RECEBIMENTO DESTE E-MAIL enviando resposta para o email sams@tre-rn.jus.br

Atenciosamente,
Flávio Roberto Guerra Seabra
{formatted_date}
                      """
            emails.append(email)
    return emails

# Chamar a função e obter os e-mails
emails_gerados = criar_emails(listas_empresas)

# Nome do arquivo de saída
output_file = 'textos_emails.txt'

# Abrir o arquivo para escrita
with open(output_file, 'w', encoding='utf-8') as file:

# Exibir os e-mails gerados
    for email in emails_gerados:
        print(email)
        file.write(email)
        print("-" * 40)
        file.write("-" * 40)
        file.write("\n")

Prezados(as) Senhores(as),

Encaminhamos a Nota de Empenho número 2024NE000332 (em anexo), à empresa L & C PARTICIPACOES E COMERCIO LTDA.
Informamos que o prazo para entrega do material é de até 30 (trinta) dias corridos, contados a partir da data de envio
desta Nota de Empenho.
Solicitamos especial atenção quanto ao PRAZO, LOCAL e HORÁRIO para entrega do material.
A entrega deverá ser realizada na Seção de Assistência Médica e Saúde Ocupacional (SAMS), localizada na Sede
Administrativa do TRE/RN. Endereço: Av. Rui Barbosa, 215. Tirol, Natal-RN. CEP: 59.015-290. Telefone: (84) 3654-
5399, nos horários de segunda a quinta-feira, das 13 às 18 horas ou na sexta-feira, das 8 às 13 horas.
FAVOR CONFIRMAR RECEBIMENTO DESTE E-MAIL enviando resposta para o email sams@tre-rn.jus.br

Atenciosamente,
Flávio Roberto Guerra Seabra
SAMS - TRE-RN, 7/10/2024.
                      
----------------------------------------
Prezados(as) Senhores(as),

Encaminhamos a Nota de Empenho número 2024NE000351 (

In [415]:
from IPython.display import display, HTML

# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Neste ponto o Objetivo 1 deve ter sido obtido</h1>'
text2 = '<h1 style="color: red;">Verifique se o arquivo "textos_emails.txt" foi criado .</h1>'

# Exibir o texto
display(HTML(text1))
display(HTML(text2))

Início de captura dos números dos itens Desertos e Fracassados

In [584]:
import re

# Definindo o padrão para capturar o trecho desejado
pattern = r'(requisitos legais foram devidamente observados.*?êxito e aguardam adjudicação)'

# Encontrar o trecho entre os padrões
match = re.search(pattern, textPDF, re.DOTALL)

if match:
    # Obter o texto completo entre os padrões
    parecer_ajdg_homolog = match.group(1).strip()  # O texto entre os dois padrões
else:
    print("Trecho não encontrado.")

In [585]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

tokens = word_tokenize(parecer_ajdg_homolog)

# Remover stopwords
stop_words = set(stopwords.words('portuguese'))
parecer_ajdg_homolog_tokenizado = [word for word in tokens if word.lower() not in stop_words]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [586]:
parecer_ajdg_homolog_tokenizado = [s for s in parecer_ajdg_homolog_tokenizado if s != ',']


In [587]:
lista_desertos = []
lista_fracassados = []
capturar = False
cont = 0
# Iterar sobre a lista e substituir 'item' por 'itens'
for i in reversed(range(len(parecer_ajdg_homolog_tokenizado))):
    if parecer_ajdg_homolog_tokenizado[i].startswith('fracass') and cont == 0:
      capturar = True
      cont = 1
    if capturar == True:
      if isinstance(parecer_ajdg_homolog_tokenizado[i], str) and parecer_ajdg_homolog_tokenizado[i].isdigit():
        lista_fracassados.append(parecer_ajdg_homolog_tokenizado[i])
      if parecer_ajdg_homolog_tokenizado[i] == 'item' or parecer_ajdg_homolog_tokenizado[i] == 'itens' :
        capturar = False
        #cont = 0

cont = 0
for i in reversed(range(len(parecer_ajdg_homolog_tokenizado))):
    if parecer_ajdg_homolog_tokenizado[i].startswith('desert') and cont == 0:
      capturar = True
      cont = 1
    if capturar == True:
      if isinstance(parecer_ajdg_homolog_tokenizado[i], str) and parecer_ajdg_homolog_tokenizado[i].isdigit():
        lista_desertos.append(parecer_ajdg_homolog_tokenizado[i])
      if parecer_ajdg_homolog_tokenizado[i] == 'item' or parecer_ajdg_homolog_tokenizado[i] == 'itens' :
        capturar = False

lista_fracassados.reverse()
print(lista_fracassados)
lista_desertos.reverse()
print(lista_desertos)

['20', '74', '84']
['22', '35', '36', '38', '40', '41', '42', '44', '45', '47', '49', '50', '52', '53', '60', '61', '63', '66', '76', '77', '83', '85', '87']


Final de captura dos números dos itens Desertos e Fracassados

listas: **lista_fracassados** e **lista_desertos**

Iniciar capturar todos os numeros dos itens do termo de referencia

In [588]:
import re

# Definindo o padrão para capturar o trecho desejado
pattern = r'(ITEM.*?O objeto desta contratação não se enquadra como sendo de bem de luxo)'

# Encontrar o trecho entre os padrões
match = re.search(pattern, textPDF, re.DOTALL)

if match:
    # Obter o texto completo entre os padrões
    tabela_produtos = match.group(1).strip()  # O texto entre os dois padrões
else:
    print("Trecho não encontrado.")

In [589]:
lista_textos_para_remover = [ "Câmara Nacional de Modelos de Licitações e Contratos da Consultoria-Geral da União",
                              "Aprovado pela Secretaria de Gestão e Inovação",
                              "UASG 70008"
                            ]

for texto in lista_textos_para_remover:
  tabela_produtos = tabela_produtos.replace(texto, "")

for i in range(1, num_pages + 1):
  tabela_produtos = tabela_produtos.replace(f"/ pg. {i}", "")

In [590]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

tokens = word_tokenize(tabela_produtos)

# Remover stopwords (palavras comuns que não têm significado relevante)
stop_words = set(stopwords.words('portuguese'))
tabela_produtos_tokenizada = [word for word in tokens if word.lower() not in stop_words]
tabela_produtos_tokenizada = [item for item in tabela_produtos_tokenizada if 'UASG' not in item]

# tabela_produtos_tokenizada

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [591]:
#Obter o número do último produto da lista

filtered_list = [s for s in tabela_produtos_tokenizada if re.match(r'^[1-9][0-9A-Z][a-zA-Z]', s)]
filtered_list = [s for s in filtered_list if not (len(s) <= 5 or 'UASG' in s)]
print(filtered_list)
ultimo = filtered_list[-1]
print(ultimo)
ultimo_produto = ''.join(filter(str.isdigit, ultimo))
ultimo_produto = int(ultimo_produto)
print(ultimo_produto)


['1Absorvente', '4Bolsa', '10Luva', '12Seringa', '13Adesivo', '14Agulha', '15Agulha', '17ARTICAÍNA', '18Lidocaína', '19MEPIVACAÍNA', '20Anestésico', '21BABADOR', '22Capa', '23Capote', '24Papel', '26Solução', '27CLOREXIDINA', '28Restaurador', '29Condicionador', '30Copo', '31Hastes', '32Cunha', '33Detergente', '35Espátula', '36Eucaliptol', '37Filme', '39Fixador', '40Flúor', '41Formocresol', '44Hidróxido', '46Indicador', '47Fita', '10mmx4mm', '48Broca', '49Broca', '50Cimento', '51Cimento', '52Arco', '54Luva', '55Luva', '56Máscara', '58Matriz', '59Lubrificante', '60Paramonoclorofenol', '61Pasta', '62Pasta', '63Pasta', '64Ponta', '66Pontas', '67Pontas', '69Resina', '70Resina', '71Resina', '72Resina', '73Resina', '74Cimento', '76Rolo', '77Rolo', '78PAPEL', '79Saco', '80Soro', '81Sugador', '82Tira', '83Verniz', '85Cureta', '86Pedra', '87INDICADOR']
87INDICADOR
87


In [593]:
# Criar a nova lista com valores de 1 a ultimo_produto, excluindo os números da lista de exclusão
lista_desertos = [int(s) for s in lista_desertos]
lista_fracassados = [int(s) for s in lista_fracassados]

# Criar lista com os produtos que não foram homologados
lista_remover = lista_desertos + lista_fracassados

# Clista com os números dos produtos homologados
produtos_homologados = [i for i in range(1, int(ultimo_produto) + 1) if i not in lista_remover] # Changed ultimo_produto to int(ultimo_produto)
print(produtos_homologados)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 39, 43, 46, 48, 51, 54, 55, 56, 57, 58, 59, 62, 64, 65, 67, 68, 69, 70, 71, 72, 73, 75, 78, 79, 80, 81, 82, 86]


Obtida a lista de produtos homologados: **produtos_homologados**

Iniciar a checagem se os produtos homologados estão nas notas de empenho

In [594]:
numeros_NE = []
for i in range (df_Notas_de_empenho.shape[0]):
  num_nota_de_empenho = df_Notas_de_empenho['NE'][i]
  integer_value = int(num_nota_de_empenho)
  numeros_NE.append(integer_value)
  numeros_NE = sorted(numeros_NE)

print(numeros_NE)

[326, 332, 333, 334, 335, 336, 348, 349, 350, 351, 352]


In [595]:
#capturar as notas de empenho para a lista notas_de_empenho_completas

import re

notas_de_empenho_completas = []

for i in range (df_Notas_de_empenho.shape[0]):
  num_nota_de_empenho = numeros_NE[i]
  # Definindo o padrão para capturar o trecho desejado
  pattern = rf'(2024 NE {num_nota_de_empenho}.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO|2024 NE {num_nota_de_empenho}.*?ANA ESMERA PIMENTEL DA FONSECA)'
  #pattern = rf'(2024 NE.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO)'
  # pattern = rf'({num_nota_de_empenho}.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO)'

  # Encontrar o trecho entre os padrões
  match = re.search(pattern, textPDF, re.DOTALL)

  if match:
      # Obter o texto completo entre os padrões
      nota_empenho = match.group(1).strip()  # O texto entre os dois padrões
      notas_de_empenho_completas.append(nota_empenho)
  else:
      print(f"Trecho não encontrado para a NE {num_nota_de_empenho}")



In [596]:
#inserir as notas de empenho tokenizadas e sem stopwords no DataFrame

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df_Notas_de_empenho['NE_Completa'] = ""

for i in range(len(notas_de_empenho_completas)):
  nltk.download('punkt')
  nltk.download('stopwords')

  # Supondo que 'text' é o conteúdo extraído do PDF
  tokens = word_tokenize(notas_de_empenho_completas[i])

  # Remover stopwords (palavras comuns que não têm significado relevante)
  stop_words = set(stopwords.words('portuguese'))
  df_Notas_de_empenho['NE_Completa'][i] = [word for word in tokens if word.lower() not in stop_words]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-596-6d0376449841>:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [597]:
#Capturar os números dos itens de cada nota de empenho e inserir no DataFrame

import numpy as np

df_Notas_de_empenho['itens_da_NE'] = ''

for i in range(df_Notas_de_empenho.shape[0]):
  num_NE = df_Notas_de_empenho['NE_Completa'][i][2]
  produtos = []
  for j in range(len(df_Notas_de_empenho['NE_Completa'][i])):
    if (df_Notas_de_empenho['NE_Completa'][i][j] == 'Item') and (df_Notas_de_empenho['NE_Completa'][i][j+1] == 'compra'):
      df_Notas_de_empenho.loc[df_Notas_de_empenho['NE'] == num_NE, 'itens_da_NE'] += df_Notas_de_empenho['NE_Completa'][i][j+3]
      df_Notas_de_empenho.loc[df_Notas_de_empenho['NE'] == num_NE, 'itens_da_NE'] += ', '
      # df_Notas_de_empenho['itens_da_NE'][i] += df_Notas_de_empenho['NE_Completa'][i][j+3]
      # df_Notas_de_empenho['itens_da_NE'][i] += ', '

# # Remove the last comma and space
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].str.rstrip(', ')

# # Split the string into a list of strings
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].str.split(', ')

# # Convert the list of strings to a list of integers
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].apply(lambda x: [int(i) for i in x])


In [598]:
for i in range(df_Notas_de_empenho.shape[0]):
  df_Notas_de_empenho['itens_da_NE'][i] = list(set(df_Notas_de_empenho['itens_da_NE'][i]))

<ipython-input-598-6d94fdba1bab>:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Notas_de_empenho['itens_da_NE'][i] = list(set(df_Notas_de_empenho['itens_da_NE'][i]))
<ipython-input-598-6d94fdba1bab>:2: FutureWarning: ChainedAssignmentEr

In [599]:
for i in range(df_Notas_de_empenho.shape[0]):
  print("----")
  print(f"Nota: {df_Notas_de_empenho['NE'][i]}")
  print(f"Produtos: {df_Notas_de_empenho['itens_da_NE'][i]}")

----
Nota: 332
Produtos: [12, 13, 14, 15, 17, 18, 19, 21, 23, 24, 28, 32, 37, 39, 43, 51, 58, 69, 75]
----
Nota: 333
Produtos: [16, 2]
----
Nota: 334
Produtos: [26]
----
Nota: 335
Produtos: [46]
----
Nota: 336
Produtos: [3, 4, 5, 6, 7, 8, 11, 25, 27, 29, 33, 34, 48, 59, 62, 64, 65, 67, 68, 70, 71, 72, 73, 80, 81, 82, 86]
----
Nota: 326
Produtos: [78]
----
Nota: 348
Produtos: [1, 79, 30, 31]
----
Nota: 349
Produtos: [9, 10, 57]
----
Nota: 350
Produtos: [54]
----
Nota: 351
Produtos: [55]
----
Nota: 352
Produtos: [56]


In [600]:
#Criar uma lista com os produtos que foram homologados mas não estão em nenhuma Nota de Empenho

produtos_homologados_nao_presentes_nas_NE = produtos_homologados.copy()

for lista in df_Notas_de_empenho['itens_da_NE']:
  for num in lista:
    if num in produtos_homologados_nao_presentes_nas_NE:
      produtos_homologados_nao_presentes_nas_NE.remove(num)

#print(f'Produtos que foram homologados mas não estão em nenhuma NE: \n{produtos_homologados_nao_presentes_nas_NE}')

In [601]:
from IPython.display import display, HTML
text1 = '<h1 style="color: red;">Aparentemente os seguintes produtos que foram homologados não estão em nenhuma Nota de Empenho</h1>'
text2 = '<h1 style="color: red;">FAVOR CHECAR!!!</h1>'
text3 = '<h1 style="color: red;">Todos os produtos homologados foram localizados em alguma Nota de Empenho</h1>'

if (len(produtos_homologados_nao_presentes_nas_NE) >0):
  # Texto em vermelho e com fonte maior
  display(HTML(text1))
  display(HTML(text2))
  print()
  print(f'Produtos que foram homologados mas não estão em nenhuma NE: \n{produtos_homologados_nao_presentes_nas_NE}')
else:
  # Texto em vermelho e com fonte maior
  display(HTML(text3))

In [602]:
from datetime import datetime

# Obter a data atual
current_date = datetime.now()

# Formatar o dia, mês e ano como strings
formatted_date = f"SAMS - TRE-RN, {current_date.day}/{current_date.month}/{current_date.year}."

if len(produtos_homologados_nao_presentes_nas_NE) > 0:
  mensagem = f"Os produtos, {produtos_homologados_nao_presentes_nas_NE}, foram homologados mas não estão em nenhuma Nota de Empenho."
else:
  mensagem = f"Todos os produtos foram homologados e localizados em alguma Nota de Empenho."

# Nome do arquivo
nome_arquivo = "mensagem_itens_homologados.txt"

# Abrindo o arquivo em modo de escrita
with open(nome_arquivo, 'w') as arquivo:
    arquivo.write(mensagem)

print(f"A mensagem foi escrita em {nome_arquivo}.")

A mensagem foi escrita em mensagem_itens_homologados.txt.


In [603]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Objetivo 2 atingido: O arquivo mensagem_itens_homologados.txt deve ter sido criado</h1>'

# Exibir o texto
display(HTML(text1))

#Início de criação da planilha de acompanhamento das entregas (Objetivo 3)

In [604]:
#Cria df_para_planilha

from datetime import datetime, timedelta

# Obter a data atual
current_date = datetime.now()

# Adicionar 30 dias à data atual
new_date = current_date + timedelta(days=30)

formatted_date = new_date.strftime("%d/%m/%Y")

resposta = input("Deseja criar a Planilha de Controle de Entregas? (s/n): ")

if resposta.lower() == 's':
  colunas_dropar = ['NE_Completa', 'itens_da_NE']
  df_para_planilha = df_Notas_de_empenho.drop(colunas_dropar, axis=1)
  df_para_planilha['Fim do prazo para entrega'] = formatted_date
  df_para_planilha['Data da Entrega'] = ''
  df_para_planilha['No. da Nota Fiscal'] = ''
  df_para_planilha['Pendências na Entrega'] = ''
  df_para_planilha['Enviado para pagamento em:'] = ''
  df_para_planilha['Pago em:'] = ''
  df_para_planilha.to_excel('Planilha de Controle de Entregas.xlsx', index=False)

Deseja criar a Planilha de Controle de Entregas? (s/n): s


In [605]:
df_para_planilha

,NE,CNPJ,Empresa,Valor,Email,Fim do prazo para entrega,Data da Entrega,No. da Nota Fiscal,Pendências na Entrega,Enviado para pagamento em:,Pago em:
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM,06/11/2024,,,,,
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",anacregis.parnamirim@gmail.com,06/11/2024,,,,,
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com,06/11/2024,,,,,
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br,06/11/2024,,,,,
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com,06/11/2024,,,,,
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",anacregis.parnamirim@gmail.com,06/11/2024,,,,,
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com,06/11/2024,,,,,
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com,06/11/2024,,,,,
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",anacregis.parnamirim@gmail.com,06/11/2024,,,,,
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM,06/11/2024,,,,,


In [607]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Objetivo 3 atingido</h1>'
text2 = '<h1 style="color: red;">Verifique se o arquivo "Planilha de Controle de Entregas.xlsx" foi criado.</h1>'

# Exibir o texto
display(HTML(text1))
display(HTML(text2))

#Início da checagem de pagamentos (Objetivo 4)

In [609]:
#fazendo split para pegar só a partir dos envios das notas de empenho

textPDF2 = textPDF.split('Envio de Nota de Empenho', 1)
if len(textPDF2) > 1:
  textPDF2 = textPDF2[1]
else:
  textPDF2 = ""


In [610]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')


# Supondo que 'text' é o conteúdo extraído do PDF
tokens = word_tokenize(textPDF2)
# Remover stopwords (palavras comuns que não têm significado relevante)
stop_words = set(stopwords.words('portuguese'))
textPDF2_tokenizado = [word for word in tokens if word.lower() not in stop_words]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [611]:
import re

lista_comprovantes = []

padrao1 = re.compile(r'^2024NE000\d{3}$')
padrao2 = re.compile(r'\d{2}[A-Z][a-z]{2}\d{2}')

for i in range (len(textPDF2_tokenizado)):
  if textPDF2_tokenizado[i] == '531814':
    comprovante = []
    for j in range (i+1, i+20):
      if padrao1.match(textPDF2_tokenizado[j]) or padrao2.match(textPDF2_tokenizado[j]):
        comprovante.append(textPDF2_tokenizado[j])
    lista_comprovantes.append(comprovante)

lista_comprovantes

lista_comprovantes_final = []
for sublist in lista_comprovantes:
  if sublist not in lista_comprovantes_final and len(sublist[1])<10:
    lista_comprovantes_final.append(sublist)


for lista in lista_comprovantes_final:
  lista[0] = lista[0][9:]

lista_comprovantes_final


[['333', '21Jun24'],
 ['334', '21Jun24'],
 ['332', '28Jun24'],
 ['336', '12Jul24'],
 ['335', '08Ago24'],
 ['335', '22Ago24']]

In [612]:
df_Notas_de_empenho = df_Notas_de_empenho.drop(columns=['NE_Completa'])

df_Notas_de_empenho['Data_Pag'] = 'Não'
df_Notas_de_empenho['NE'] = df_Notas_de_empenho['NE'].astype(str)

for i in range (len(lista_comprovantes_final)):
  for j in range (df_Notas_de_empenho.shape[0]):
    if lista_comprovantes_final[i][0] == df_Notas_de_empenho['NE'][j]:
      df_Notas_de_empenho['Data_Pag'][j] = lista_comprovantes_final[i][1]

df_Notas_de_empenho

<ipython-input-612-ea290ebfdcf2>:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Notas_de_empenho['Data_Pag'][j] = lista_comprovantes_final[i][1]
<ipython-input-612-ea290ebfdcf2>:9: FutureWarning: ChainedAssignmentError: behaviour will c

,NE,CNPJ,Empresa,Valor,Email,itens_da_NE,Data_Pag
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM,"[12, 13, 14, 15, 17, 18, 19, 21, 23, 24, 28, 3...",28Jun24
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",anacregis.parnamirim@gmail.com,"[16, 2]",21Jun24
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com,[26],21Jun24
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br,[46],22Ago24
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com,"[3, 4, 5, 6, 7, 8, 11, 25, 27, 29, 33, 34, 48,...",12Jul24
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",anacregis.parnamirim@gmail.com,[78],Não
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com,"[1, 79, 30, 31]",Não
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com,"[9, 10, 57]",Não
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",anacregis.parnamirim@gmail.com,[54],Não
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM,[55],Não


In [613]:
df_Notas_de_empenho.to_excel('Planilha de Controle de Pagamentos.xlsx', index=False)

In [614]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Objetivo 4 atingido</h1>'
text1 = '<h1 style="color: red;">Verifique se o arquivo "Planilha de Controle de Pagamentos.xlsx" foi criado.</h1>'

# Exibir o texto
display(HTML(text1))